In [6]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("setup").getOrCreate()

In [17]:
import csv
import re

def process_csv(input_file, output_file):
    # Define a function to check if value is a date in format DD/MM/YYYY
    def is_date(s):
        pattern = '^\d{1,2}(\/|\-)\d{1,2}(\/|\-)\d{4}$'
        return bool(re.match(pattern, s))
    
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        for i, row in enumerate(reader):
            # Check the fifth value of the row, if exists
            if not is_date(row[4]) and i > 0 and str(row[4]).lower() != 'null':
                row[3] = str('') + str(row[3]) + ',' + str(row[4]) + str('')
                row.pop(4)
                writer.writerow(row)
            else:
                writer.writerow(row)

# Example usage:
process_csv('/home/jovyan/clean_me.csv', 'cleaned.csv') 

In [20]:
def read_csv(input_path):

    # Read the CSV into a DataFrame
    df = spark.read.option("delimiter", ",").option("quote", "\"").csv(input_path, header=True, inferSchema=True)

    return df

# Example usage:
df = read_csv('cleaned.csv')
df.show()

+--------+----------------+--------+------+------------+-------------------+--------------------+--------+-----+----+
|order_id|delivery_company|quantity| price|ordered_date|            address|                 _c6|     _c7|  _c8| _c9|
+--------+----------------+--------+------+------------+-------------------+--------------------+--------+-----+----+
|       1| delivery_comp_1|       1|245,52|    9/2/2022| Cedar Lane Houston|            CA 90001|    null| null|null|
|       2| delivery_comp_2|       2|114,77|        null|        Main Street|   New York CA 60601|    null| null|null|
|       3| delivery_comp_3|    null|739,43|   14-3-2022|        Main Street|    Chicago TX 10001|    null| null|null|
|       4| delivery_comp_0|       1|878.93|   20/4/2022|         Oak Avenue|Los Angeles FL 90001|    null| null|null|
|       5| delivery_comp_1|       2|481,44|        null|Maple Drive Chicago|            FL 60601|    null| null|null|
|       6| delivery_comp_2|     #NA| 78,13|        null|